In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import json5
import pandas as pd
import numpy as np
import requests
import time
import pymongo
from datetime import datetime
from scipy import stats
from scipy.stats import linregress
from citipy import citipy

In [2]:
from config import EV_Api_key


In [3]:
#Create the Query URL to

query_url= "https://developer.nrel.gov/api/alt-fuel-stations/v1.json?fuel_type=ELEC&ev_network="
api_key = "&api_key="

#Create the Variables to hold the EVTypes and Collections that will be used in the Mongo DB
evtypes = ["Tesla","Volta","Greenlots","FLO"]
collections=["tesla_stations_us","volta_stations_us","greenlots_stations_us","flo_stations_us"]

# Connect to the MongoDB
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Declare the Mongo Database
db = client.EV_db

# Call the API and Loop through each EV Provider to create the Collections
for (evtype,collection) in zip(evtypes,collections):
    results =[]
    try:
        response = requests.get(query_url + evtype + api_key+EV_Api_key).json()
        
        for station in response['fuel_stations']:

            EV_dict = {
                "access_code":station['access_code'],
                "facility_type":station['facility_type'],
                "ev_pricing":station['ev_pricing'],
                "ev_connector_types":station['ev_connector_types'][0],
                "access_days_time":station['access_days_time'],
                "open_date":  station['open_date'],
                "latitude":station['latitude'],
                "longitude":station['longitude'],
                "street_address":station['street_address'],
                "city":station['city'],
                "zip":station['zip'],
                "state":station['state']

            }
            results.append(EV_dict)
       

    except:
        print('info not found')
        
    # Declare the collection

    stations = db[collection]
 
    # Insert a document into the collection
    for document in results:
        stations.insert_one(document) 